In [1]:
import numpy as np
from docplex.mp.model import Model
from itertools import product

##### Importando e tratando instância ######

caminho = 'instancias/'
instancia = 'Inst01.txt'
#instancia = 'Inst02.txt'
arquivo = caminho+instancia

with open(arquivo, 'r') as file:
    n, m = [int(v) for v in file.readline().split()] # n trens, m trechos
    O = [[int(v) for v in file.readline().split()] for i in range(n)]
    
μ = [[O[j][2 * s] for s in range(int(len(O[j])/2))] for j in range(n)]
p = [[O[j][2 * s+1] for s in range(int(len(O[j])/2))] for j in range(n)]    

maq = []
for i in range(m):
    maq.append([])
    for j in range(len(μ)):
        if i in μ[j]:
            maq[i].append(j)
            
pdict = {}
for i in range(len(μ)):
    for j in range(len(μ[i])):
        pdict[(i, μ[i][j])]=p[i][j]
        
##### Gerando o Big-M #####

M = 0
for i in range(len(p)):
    eminho = sum(p[i])
    M = M+eminho    
M = M**2

##### Criando Modelo #####

mdl = Model(name = 'JobShopTrain')

##### Criando variáveis de decisão ######

xis = []
for i in range(len(μ)):
    for j in enumerate(μ[i]):
        xis.append((i,j[1]))
        
dablio = []
for j in range(len(maq)):
    for i in enumerate(maq[j]):
        for k in enumerate(maq[j]):
            if k > i:
                dablio.append((i[1],k[1],j))

x = mdl.integer_var_dict(xis, lb=0, ub=M, name='x') # variável x_ij
w = mdl.binary_var_dict(dablio, lb=0, ub=1, name='w') # variável w_ikj

##### Criando função objetivo #####

cmax = mdl.continuous_var(0, M, 'Cmax') # variável da função objetivo
for i in range(n):
    for j in range(n):
        mdl.add(cmax >= x[i, μ[i][j]] + p[i][j]) # equação (2)
mdl.minimize(cmax) # equação (1) => função objetivo

###### Adicionando restrições ######

for (i, j, l) in product(range(n), range( len(μ[i]) ), range( len(μ[i]) ) ): # verificar isso
    if j == l+1:
        mdl.add_constraint(x[i, μ[i][j]] == x[i, μ[i][l]]+p[i][l]) # equação (3) modificada "no-wait"

for (i, k) in product(range(n), range(n)):
    if k > i:
        for j in range(m):
            if (i, j) in x and (k, j) in x:
                mdl.add_constraint(x[i, j] + pdict[(i,j)] <= x[k, j]+M*(1-w[i, k, j])) # equação (4)
                mdl.add_constraint(x[k, j] + pdict[(k,j)] <= x[i, j]+M*w[i, k, j]) # equação (5)

##### Limitando o tempo de execução ######

mdl.set_time_limit(1*60*60) 

##### Exportando modelo para *.lp ######

mdl.export_as_lp(basename="modelo_lp", path="Modelo")

###### Resolvendo e gerando o relatório final ######

mdl.solve()
status = mdl.solve_details.status
print("Status: ", status)

print("Número de Variáveis: ", mdl.number_of_variables)  
print("Número de Restrições: ", mdl.number_of_constraints)
print("Tempo de execução (s): %.4f" %mdl.solve_details.time)
print("Nós B&B: ", mdl.solve_details.nb_nodes_processed)
print("Função-objetivo: ", mdl.objective_value)
print("Best Bound: %.4f" %mdl.solve_details.best_bound)            
print("GAP: %.4f" %mdl.solve_details.mip_relative_gap) 

print()
print("Lista de Alocação ([trem, trecho], início, fim):")
aux = []        
for i in range(n):
    for j in range(n):
        aux.append(([i, μ[i][j]], x[i, μ[i][j]].solution_value, x[i, μ[i][j]].solution_value+pdict[(i,μ[i][j])]))        
solOrd = sorted(aux, key=lambda tup: tup[1])
print(solOrd)
print()

Status:  integer optimal solution
Número de Variáveis:  15
Número de Restrições:  25
Tempo de execução (s): 1.0930
Nós B&B:  0
Função-objetivo:  169.99999999999403
Best Bound: 170.0000
GAP: 0.0000

Lista de Alocação ([trem, trecho], início, fim):
[([1, 1], 0, 40), ([2, 4], 20.0, 70.0), ([1, 2], 40.0, 70.0), ([0, 0], 70.0, 100.0), ([1, 4], 70.0, 120.0), ([2, 2], 70.0, 100.0), ([0, 2], 100.0, 130.0), ([2, 0], 100.0, 150.0), ([0, 3], 130.0, 170.0)]

